In [1]:
from kafka import KafkaProducer,KafkaConsumer
from datetime import datetime
import time
import threading, Queue
from time import sleep

In [2]:
topics = ["beatsGlobal","beatsLocal"]
KFC_0 = ['smartaqnet-0-node1.teco.edu:9092','smartaqnet-0-node2.teco.edu:9092','smartaqnet-0-node3.teco.edu:9092']
KFC_0 = ['129.13.170.147:9092','129.13.170.157:9092','129.13.170.150:9092']

KFC_1 = ['smartaqnet-1-node1.teco.edu:9092']
KFC_0 = ['129.13.170.133:9092']

In [3]:


def heartBeatSender(KFC_0, KFC_1, topics, heartBeatQ0 , heartBeatQ1, heartBeatQ2, heartBeatQ3):
    producer_KFC_Main = KafkaProducer(bootstrap_servers= KFC_0, batch_size = 0)
    # Send heart beats 
    producer_KFC_Backup = KafkaProducer(bootstrap_servers= KFC_1, batch_size = 0)
    while(True):


        heartBeat = str(datetime.now())
        # The ordering of the messages is guaranteed
        heartBeatQ0.put(heartBeat)
        heartBeatQ1.put(heartBeat)
        heartBeatQ2.put(heartBeat)
        heartBeatQ3.put(heartBeat)

        print(heartBeat)

        producer_KFC_Main.send(topic = topics[0], value = heartBeat)

        producer_KFC_Main.send(topic = topics[1], value = heartBeat)

        producer_KFC_Backup.send(topic = topics[1], value = heartBeat)

        producer_KFC_Main.flush()
        producer_KFC_Backup.flush()
        sleep(2)
    


def heartBeatListener(KFC, topic, heartBeatQ, receivedQ ,msgType):
    
    heartBeat = heartBeatQ.get()
    
    #print(heartBeat)
    
    heartBeat = datetime.strptime(heartBeat, '%Y-%m-%d %H:%M:%S.%f')
    
    consumer_KFC = KafkaConsumer(bootstrap_servers= KFC,
                                 auto_offset_reset='latest')
    consumer_KFC.subscribe(topic)
    
    for message in consumer_KFC:
        if(heartBeat > datetime.strptime(str(message.value), '%Y-%m-%d %H:%M:%S.%f')):
            print("NNN")
            continue
        else:
            aStr = "Received:{} - {}".format(msgType,str(message.value))
            
            receivedQ.put([msgType, aStr])
            
            heartBeat = heartBeatQ.get()
    
            heartBeat = datetime.strptime(heartBeat, '%Y-%m-%d %H:%M:%S.%f')
        
        sleep(1)






In [4]:
# Define the Queues
heartBeatQ0 = Queue.Queue()
heartBeatQ1 = Queue.Queue()
heartBeatQ2 = Queue.Queue()
heartBeatQ3 = Queue.Queue()
receivedQ = Queue.Queue()

In [5]:
# Define the threads
threadHeartBeatSender = threading.Thread(target=heartBeatSender,args=(KFC_0, KFC_1, topics, heartBeatQ0, heartBeatQ1,heartBeatQ2,heartBeatQ3))
threadHeartBeatReceiver0 = threading.Thread(target=heartBeatListener,args=(KFC_0, topics[0], heartBeatQ0, receivedQ,0))
threadHeartBeatReceiver1 = threading.Thread(target=heartBeatListener,args=(KFC_1, topics[0], heartBeatQ1, receivedQ,1))
threadHeartBeatReceiver2 = threading.Thread(target=heartBeatListener,args=(KFC_0, topics[1], heartBeatQ2, receivedQ,2))
threadHeartBeatReceiver3 = threading.Thread(target=heartBeatListener,args=(KFC_1, topics[1], heartBeatQ3, receivedQ,3))


In [6]:
threadHeartBeatSender.start()



2018-07-22 18:25:27.314788
2018-07-22 18:25:29.915242


In [7]:
threadHeartBeatReceiver0.start()
threadHeartBeatReceiver1.start()
threadHeartBeatReceiver2.start()
threadHeartBeatReceiver3.start()

2018-07-22 18:25:31.923539
2018-07-22 18:25:33.944955


In [8]:
res = [0,0,0,0]
while(True):
    
    for i in range(60):
        aMsg = receivedQ.get()
        res[aMsg[0]] += 1
        print(res)
    if(max(res) - min(res) < 5):
        print("OK")
        aMin = min(res)
        for j in range(len(res)):
            res[j] -= aMin
    else:
        print(res)
    
        

[0, 0, 0, 1]
[0, 1, 0, 1]
[1, 1, 0, 1]
[1, 1, 1, 1]
[1, 1, 1, 2]
[1, 1, 2, 2]
[1, 1, 3, 2]
[2, 1, 3, 2]
[2, 2, 3, 2]
[2, 2, 3, 3]
[2, 2, 3, 4]
[2, 2, 4, 4]
2018-07-22 18:25:35.965161
[3, 2, 4, 4]
[3, 3, 4, 4]
[3, 3, 5, 4]
[3, 3, 5, 5]
2018-07-22 18:25:37.976942
[3, 4, 5, 5]
[4, 4, 5, 5]
NNN
NNN
[4, 4, 6, 5]
[4, 4, 6, 6]
2018-07-22 18:25:39.991475
[4, 5, 6, 6]
[5, 5, 6, 6]
NNN
NNN
[5, 5, 7, 6]
[5, 5, 7, 7]
2018-07-22 18:25:42.000805
[6, 5, 7, 7]
[6, 6, 7, 7]
NNN
NNN
[6, 6, 8, 7]
[6, 6, 8, 8]
2018-07-22 18:25:44.018487
[7, 6, 8, 8]
[7, 7, 8, 8]
NNNNNN

[7, 7, 8, 9]
[7, 7, 9, 9]
2018-07-22 18:25:46.029342
[8, 7, 9, 9]
[8, 8, 9, 9]
NNNNNN

[8, 8, 9, 10]
[8, 8, 10, 10]
2018-07-22 18:25:48.044405
[8, 9, 10, 10]
[9, 9, 10, 10]
NNN
[9, 9, 11, 10]
NNN
[9, 9, 11, 11]
2018-07-22 18:25:50.059565
[9, 10, 11, 11]
[10, 10, 11, 11]
NNNNNN

[10, 10, 12, 11]
[10, 10, 12, 12]
2018-07-22 18:25:52.073777
[10, 11, 12, 12]
[11, 11, 12, 12]
NNN
NNN
[11, 11, 13, 12]
[11, 11, 13, 13]
2018-07-22 18:25:54.086305


KeyboardInterrupt: 

2018-07-22 18:27:22.689486


In [ ]:
receivedQ.get()

2018-07-18 17:19:34.360249


In [8]:
res = [0,1,2,3]

In [9]:
threadHeartBeatSender.
threadHeartBeatReceiver0.start()
threadHeartBeatReceiver1.start()
threadHeartBeatReceiver2.start()
threadHeartBeatReceiver3.start()

3